# TP 1 et 2 : Accès aux données avec index 
SAM, sujet pour étudiants

date de modification : 01/02/2023

NOM: VIN

Prénom: Charles

Objectifs:
Savoir organiser des données en pages pour permettre de modifier un tuple en ne modifiant qu'une seule page.

Comprendre les méthodes d'accès suivantes :

*   Lecture séquentielle d'une fichier : "table access full"
*   Lecture d'un tuple dont on connait le rowid : "table access by index rowid"
*   Opération de sélection par lecture séquentielle et filtrage 

Comprendre les méthodes d'indexation :

*   Créer un index
*   Opération de Sélection par index et lecture par rowid

Mise à jour de données
*   Sélectionner un tuple et modifier un de ses attributs
*   Modifier l'index en conséquence lorsque l'attibut modifié est indexé

Persistence
*   Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement
*   Adapter en conséquence les opérations de modification de l'index


In [2]:
import os
import shutil as sh
import numpy as np
import random
from random import choice
from string import ascii_lowercase
import time

# le nom de la table 
TABLE = "T"
print("nom de la table :", TABLE)


# le nom du fichier qui contient les données de la table
def nom_fichier(table):
    return table + ".csv"

nom de la table : T


# Générer un fichier

Création du fichier contenant un exemple de données.
Ce sont des données au format csv. On suppose que chaque ligne correspond à un tuple d'une table **T** ayant *n* attributs :

$$ T (a_0, a_1, a_2, ..., a_{n-1})$$

Le premier attribut $a_0$ est unique

Les attributs suivants ne sont pas uniques (ils ont des doublons). Pour l'attribut $a_1$, il y a en moyenne 2 tuples par valeur, pour $a_2$ il y a en moyenne 4 tuples par valeurs, etc. 

Les attributs sont des nombres entiers sauf le dernier qui est une chaine de caractères.


In [3]:
# dure environ 40s pour 5M lignes

nb_lines = 5 * 1000 # * 1000
nb_attributes = 7

longueur_dernier_attribut = 100
# attribut_chaine_caracteres = "".join(choice(ascii_lowercase) for i in range(longueur_dernier_attribut))
attribut_chaine_caracteres = ''.join('-' for i in range(longueur_dernier_attribut))
# print("le dernier attribut de chaque tuple est la chaine de caracètes :", attribut_chaine_caracteres)

nb_valeurs_distinctes = nb_lines


# le premier attribut est unique
a = [random.sample(range(nb_valeurs_distinctes), nb_lines)]

# les attributs suivants ont des domaines plus petits
for i in range(1, nb_attributes):
  nb_valeurs_distinctes = max(2, int(nb_valeurs_distinctes / 2))
  a.append(np.random.randint(0, nb_valeurs_distinctes, nb_lines))

# on concatène "verticalement" les colonnes pour former les tuples.
# Le dernier attribut est une chaine de caractères
b = [ ','.join(map(lambda x: str(x), e)) + f",{attribut_chaine_caracteres}\n" for e in zip(*a)]

# on stocke les données dans un fichier
fichier = nom_fichier(TABLE)
with open(fichier, "w") as f:
  f.write(''.join(b))

print("données créées dans le fichier :", fichier)

données créées dans le fichier : T.csv


On affiche le début et la fin du fichier et son nombre de lignes ( = card(T))

In [4]:
%%bash -s "$fichier"
echo "debut de $1 :"
head -n 2 $1
echo
echo "fin de $1 : "
tail -n 2 $1
echo
echo "size (lines) :"
wc -l $1

debut de T.csv :
4701,149,412,175,267,41,67,----------------------------------------------------------------------------------------------------
1758,1871,185,19,133,151,39,----------------------------------------------------------------------------------------------------

fin de T.csv : 
2808,381,992,512,112,3,73,----------------------------------------------------------------------------------------------------
3900,971,948,164,32,39,33,----------------------------------------------------------------------------------------------------

size (lines) :
5000 T.csv


# Lecture séquentielle

On définit un *iterateur* pour lire séquentiellement chaque ligne de la table stockée entièrement dans un seul fichier. Le mot python *yield* facilite la définition d'un itérateur.

Cet itérateur est invoqué pour lire la table et appliquer un filtre.


In [5]:
def lecture_sequentielle(table):
  fichier = nom_fichier(table)
  with open(fichier, "r") as f:
    for i, line in enumerate(f):
      yield i, line

def filtrer_table(table, valeur_recherchee):
  for i, line in lecture_sequentielle(table):
      a = int(line.split(',')[0])
      if a == valeur_recherchee :
        print(f"ligne {i} :", line.strip())



s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_table(TABLE, s)
print("done in", round(time.time() - t1, 3), "s")

valeur recherchée : 31
ligne 3850 : 31,1884,111,538,139,54,44,----------------------------------------------------------------------------------------------------
done in 0.013 s


# Découper une table en pages

On organise les données en pages. 
Pour faciliter le TP, chaque page est représentée par un "petit" fichier mais en réalité une page serait un bloc d'un fichier.

Dans la suite du TP, on accédera toujours aux pages.
Le fichier créé initialement, contenant tous les tuples, ne sera plus utilisé.

In [6]:
def page_dir_name(table):
  return table + "_pages"

print("les pages sont stockées dans le dossier", page_dir_name(TABLE) )

def decoupe_table_en_pages(table, nb_tuple_par_page):
  page_dir = page_dir_name(table)

  # vider le dossier qui contiendra les pages
  if(os.path.exists(page_dir)):
    sh.rmtree(page_dir)
  os.makedirs(page_dir, exist_ok=True)

  # lire le fichier contenant tous les tuples
  p=0
  lines = []
  for i, line in lecture_sequentielle(table):
    lines.append(line)
    if (i+1) % nb_tuple_par_page == 0:
      
      # créer une page
      p += 1
      with open(page_dir + f"/page{p}", "w") as fp:
        fp.write(''.join(lines))
      lines = []

  # créer une dernière page, si nécessaire
  if len(lines) > 0:
    p +=1
    with open(page_dir + f"/page{p}", "w") as fp:
        fp.write(''.join(lines))

  print("nb pages créées :", p)


decoupe_table_en_pages(TABLE, nb_tuple_par_page=1000)

les pages sont stockées dans le dossier T_pages
nb pages créées : 5


Afficher le nombre de tuples dans une page (pour quelques pages)

In [7]:
%%bash -s "$TABLE"
wc -l $1_pages/* | head -n 3

  1000 T_pages/page1
  1000 T_pages/page2
  1000 T_pages/page3


# Lecture séquentielle d'une table découpée en pages

In [8]:
def lecture_sequentielle_par_page(fichier):
#   # a faire : pour chaque page, lire ses lignes
#   # une ligne devient un tuple
#   # retourner un itérateur contenant le numéro de page, la position dans la page et le tuple
    page_dir = page_dir_name(fichier)
    nb_pages = len(os.listdir(page_dir))
    for page_nb in range(1, nb_pages+1):
        with open(page_dir + "/page" + str(page_nb), "r") as page:
            for i, line in enumerate(page):
                yield [page_nb, i, tuple(line.split(","))]


def filtrer_fichier_par_pages(fichier, valeur_recherchee):
    l = lecture_sequentielle_par_page(fichier)
    print('Research started')
    for page_nb, pos_page, line in l:
        if int(line[0]) == valeur_recherchee:
            print(f"Page n°{page_nb}, Line n°{pos_page} :", line)

#   # a faire pour chaque (numéro de page, position dans la page, tuple) obtnenu en invoquan la méthode ci dessus
#   # convertir le 1er attribut en un nombre l'afficher si il est egal à la valeur recherchee



s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_fichier_par_pages(TABLE, s)
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 11
Research started
Page n°3, Line n°940 : ('11', '2449', '164', '219', '154', '9', '54', '----------------------------------------------------------------------------------------------------\n')
done in 0.03 s


# Lecture d'un tuple dans une page

Cette fonction retourne un tuple

In [9]:
def lecture_tuple(table, num_page, position):
    page_dir = page_dir_name(table)
    with open(page_dir + '/page' + str(num_page), 'r') as page:
        return page.readlines()[position]


t1 = time.time()
print(lecture_tuple(TABLE, 123, 456))
print("done in", round((time.time() - t1)*1000, 1), "ms")

FileNotFoundError: [Errno 2] No such file or directory: 'T_pages/page123'

# Créer un index

## Créer un index unique pour l'attribut $a_0$

On sait que $a_0$ est unique. 
Une entrée associe une *clé* à une *valeur*.
*   La *clé* est la valeur du 1er attribut. 
*   La *valeur* est un **rowid** formé des informations (page, position)



In [10]:
def creation_index_unique(table):
    index = {}
    page_dir = page_dir_name(table)
    nb_pages = len(os.listdir(page_dir))
    for page_nb in range(1, nb_pages+1):
        with open(page_dir + "/page" + str(page_nb), "r") as page:
            for i, line in enumerate(page):
                index[int(line.split(",")[0])] = (page_nb, i)
    return index
#    # la clé est la valeur du 1er attribut
#    # la valeur est un rowid composé de (page, position)

t1 = time.time()
INDEX_UNIQUE_a0 = creation_index_unique(TABLE)
print("done in", round(time.time() - t1, 3), "s")

done in 0.015 s


In [11]:
# vérifier l'index unique
s = np.random.randint(nb_valeurs_distinctes)
print(s, INDEX_UNIQUE_a0[s])

2 (3, 136)


## Créer un index non unique pour l'attribut $a_i$

On donne un nom de table et le numéro $i$ de l'attribut $a_i$ de la table.

In [12]:
def creation_index(table, numero_attribut):
    index = {}
    page_dir = page_dir_name(table)
    nb_pages = len(os.listdir(page_dir))
    for page_nb in range(1, nb_pages+1):
        with open(page_dir + "/page" + str(page_nb), "r") as page:
            for i, line in enumerate(page):
                try:
                    index[int(line.split(",")[numero_attribut])].append((page_nb, i))
                except KeyError:
                    index[int(line.split(",")[numero_attribut])] = [(page_nb, i)]
    return index
    # la clé est la valeur du 1er attribut <= ???? Du premier ou du ième ??  
    # la valeur est une *liste* de rowid

t1 = time.time()
INDEX_a2 = creation_index(TABLE, 2)
print("done in", round(time.time() - t1, 3), "s")

done in 0.016 s


In [13]:
# vérifier l'index
s = np.random.randint(nb_valeurs_distinctes/4)
print("valeur recherchée :", s)
for r in INDEX_a2[s]:
    print(r)

valeur recherchée : 18
(2, 482)
(4, 183)
(4, 901)
(5, 716)
(5, 721)
(5, 942)


# Accès par index

## Accès ciblé 



### Index unique scan. 
Accès pour rechercher le tuple dont l'attribut indexé a une valeur donnée (on suppose que l'attribut est unique)

In [14]:
def acces_par_index_unique(index_unique, table, valeur_recherchee):
    return index_unique[valeur_recherchee]

s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
acces_par_index_unique(INDEX_UNIQUE_a0, TABLE, s)
print("done in", round((time.time() - t1)*1000, 2), "ms")

valeur recherchée : 22
done in 0.15 ms


### Index scan
Accès pour rechercher les tuples dont l'attribut indexé a une valeur donnée. On suppose que l'attribut n'est pas unique et que plusieurs tuples sont retrouvés

In [19]:
def acces_par_index(index, table, valeur_recherchee):
    return index[valeur_recherchee]


s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
acces_par_index(INDEX_UNIQUE_a0, TABLE, s)
print("done in", round((time.time() - t1)*1000, 2), "ms")

valeur recherchée : 37
done in 0.11 ms


## Accès par intervalle 
Index range scan



### Accès par intervalle sur un attribut unique
Accès pour rechercher les tuples dont l'attribut indexé est unique et a une valeur comprise dans un intervalle donné.

In [24]:
def acces_intervalle_par_index_unique(index_unique, table, fichier, borne_inf, borne_sup):
    tuple_list = []
    for key in index_unique.keys():
        if borne_inf < key and key < borne_sup:
            tuple_list.append(index_unique[key])
    return tuple_list

s = np.random.randint(nb_valeurs_distinctes/4)
print("valeur recherchée :", s)

t1 = time.time()
acces_intervalle_par_index_unique(INDEX_UNIQUE_a0, TABLE, s, 10, 15)
print("done in", round(time.time() - t1, 2), "s")
acces_intervalle_par_index_unique(INDEX_UNIQUE_a0, TABLE, s, 10, 15)

valeur recherchée : 13
done in 0.03 s


[(1, 50), (3, 940), (4, 702), (5, 728)]

### Accès par intervalle sur un attribut NON unique
Accès pour rechercher les tuples dont l'attribut indexé n'est **pas** unique et a une valeur comprise dans un intervalle donné.

In [28]:
def acces_intervalle_par_index(index, table, fichier, borne_inf, borne_sup):
    tuple_list = []
    for key in index.keys():
        if borne_inf < key and key < borne_sup:
            for rowid in index[key]:
                tuple_list.append(rowid)
    return tuple_list



s = np.random.randint(nb_valeurs_distinctes / 4)
print("valeur recherchée :", s)

t1 = time.time()
acces_intervalle_par_index(INDEX_a2, TABLE, s, 10, 15)
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 2
done in 0.0 s


# Mise à jour de données




## Sélectionner un tuple et modifier un de ses attributs



### Modification d'un seul tuple

On donne une valeur *v* de l'attribut clé $a_0$. Ajouter 1 à l'attribut $a_1$. Cela correspond à l'instruction

update T
set a1 = a1+1
where a0 = *v*

Après la modification, accéder aux données pour vérifier que le tuple a bien été modifié. Par exemple, invoquer la fonction
acces_par_index_unique(index, table, v)



In [18]:
def ecriture_tuple(table, num_page, position, attribut, new_valeur):
    page_dir = page_dir_name(table)
    with open(page_dir + '/page' + str(num_page), 'rw') as page:
        line = page.readlines()[position]
        splited_line = line.split(',')
        splited_line[attribut] = str(new_valeur)
        ','.join(splited_line)


def incremente(index, table, v):
    page_nb, line = index[v]
    


incremente(INDEX_UNIQUE_a0, TABLE, s)
acces_par_index_unique(INDEX_UNIQUE_a0, TABLE, s)

<div style="color: red">PROBLEME ICI</div>

### Modification de plusieurs tuples

On donne une valeur *v* de l'attribut $a_2$ qui n'est pas unique. Ajouter 1 à l'attribut $a_3$.

update T set a3 = a3+1 where a1=v



## Modifier l'index en conséquence lorsque l'attibut modifié est indexé
Par exemple, si $a_3$ est indexé, la mise à jour de la question précédente implique d'actualiser l'index sur $a_3$  pour que les rowid associés à l'ancienne valeur de $a_3$  soient associés à la nouvelle valeur de $a_3$ .

In [ ]:
def update_index(index, old_val, new_val):
    value = index[old_val]
    del index[old_val]
    index[new_val] = value

# Persistence

Dans cette partie, on veut rendre les index persistents en les stockant dans des pages. Cela permet d'utiliser les index plus efficacement sans les recréer entièrement.

## Stockage d'un index unique

Proposer une solution pour stocker les entrées triées d'un index unique dans plusieurs pages avec une taille de page fixée (500 entrées par page,  soit 500 clés + 500 rowid).

## Stockage d'un index non unique
Proposer une solution pour stocker les entrées triées d'un index non unique dans plusieurs pages avec une taille de page fixée. Dans une page, le total du nombre  de clés + le nombre de rowid ne peut pas dépasser 1000.

## Adapter en conséquence les opérations de modification de l'index

# Questions facultatives

## Index bitmap
*   Proposer un index ayant une structure "bitmap" pour l'attribut $a_5$. Idem pour l'attribut $a_6$. 
*   En utilisant les 2 index bitmap, rechercher les tuples de T tels que $a_5 = v_1$ et $a_6 = v_2$ pour deux valeurs $v_1, v_2$ appartenant au domaine de $a_5 \cap a_6$ .

## Index couvrant une requête
* Illustrer les cas vus en TD.
